<a href="https://colab.research.google.com/github/RyanTokManMokMTM/NLP_Training4/blob/main/Words2Vec.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!unzip wiki_zh_2019
#unzip the zipfile

In [ ]:
from os import listdir
from os.path import isfile, isdir, join
import json
ALL_DATAS = []
filesList = {
    0:"AA",
    1:"AB",
    2:"AC",
    4:"AD",
    5:"AE",
    6:"AF",
    7:"AG",
    8:"AH",    
    9:"AI",
    10:"AJ",
    11:"AK",
    12:"AL",
    13:"AM"
}

for i in filesList:
  folderPath = "./wiki_zh/%s/" % filesList[i]
  files = listdir(folderPath)
  for i in files:
    curFilePath = folderPath + "%s"%i 
    print(curFilePath)
    with open(curFilePath) as f:
      for j in f.readlines():
        str2Json = json.loads(j)
        text = "".join(str2Json["text"].strip().split())
        ALL_DATAS.append(text)
      f.close()


In [ ]:
!pip install opencc-python-reimplemented

In [ ]:
#convert text 
from opencc import OpenCC
#convert to zh-hk
ccConvertor = OpenCC("s2hk")
transChineseList = []
for i in ALL_DATAS:
  transChineseList.append(ccConvertor.convert(i))


In [ ]:
#store convertd data to csv
import pandas as pd
datas = pd.DataFrame(transChineseList)
datas.to_csv("alldatas.csv",encoding="utf-8")

# cut words

In [ ]:
!pip install jieba

In [ ]:
#load stopwords list
import codecs 
fp = codecs.open("./stopwords.dat","r",encoding="utf-8")
contents = fp.read()
fp.close()

In [ ]:
import jieba
import jieba.analyse
cutWordsListDatas = []
x = 0;
for i in transChineseList:
  cutWordsListDatas.append(" ".join([word for word in jieba.cut(i,cut_all=False) if word not in contents]))
  if(x%5000 == 0):
    print(x)
  x+=1

In [45]:
cutWordsTxt = "cutWordsList2.txt"
with open(cutWordsTxt,"w",encoding="utf-8") as fw:
  for data in cutWordsListDatas:
    fw.write(data)
    fw.write("\n")


# Word2vec for our text

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [6]:
#store txt to array\
cutWordsListDatas = []
with open("/content/drive/MyDrive/cutWordsList.txt") as f:
  for i in f.readlines():
    cutWordsListDatas.append(i.rstrip("\n"))

In [ ]:
!pip install word2vec

In [46]:
import word2vec
word2vec.word2vec("cutWordsList2.txt","cutWordsList2.bin",size=300,min_count=1,verbose=True,threads=8,binary=1)

Running command: word2vec -train cutWordsList2.txt -output cutWordsList2.bin -size 300 -window 5 -sample 1e-3 -hs 0 -negative 5 -threads 8 -iter 5 -min-count 1 -alpha 0.025 -debug 2 -binary 1 -cbow 1
Starting training using file cutWordsList2.txt


Exception: ignored

In [ ]:
model = word2vec.load("cutWordsListTest.bin")

In [39]:
temp = model.similar("北京")

In [42]:
for i in temp[0]:
  print(model.vocab[i],model)

上海 148
下半年 9268
第四 773
教務長 17331
院長 1704
撤消 9341
月間 10072
工程局 13684
教育部 1067
中國國家 3638
